### Práctica de Web Scraping

Se va a considerar una de las páginas vista en la teoría que contiene fotos de trenes:https://www.vialibre-ffe.com/multi_foto.asp?cs=mult. En esta página aparece un menú con varios grupos de fotos de trenes. A su vez, cada grupo contiene un conjunto de subgrupos de fotos. Por ejemplo, el grupo __Renfe operadora__ se encuentra en la página https://www.vialibre-ffe.com/multi_ind_fotos.asp?cat=mu01.

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')

Dentro de este grupo por ejemplo se encuentra el subgrupo __Serie120 en Irún-Hendaya__ en la página https://www.vialibre-ffe.com/multi_galeria.asp?gal=524. 

In [ ]:
from IPython.display import Image
Image(filename='Captura2.png')

En dicho subgrupo se encuentran las imágenes de dicho subgrupo. Por ejemplo, a continuación, se muestra una de las imágenes del subgrupo __Serie120 en Irún-Hendaya__

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')
#for i in range(1,4):
#    Image(filename='Captura'+str(i)+'.png')

Las imágenes contenidas en dicha página se pueden encontrar en las etiquetas __&lt;li> &lt;img ...__ que son hijas de la etiqueta __&lt;ul class="pgwSlideshow">__. Por ejemplo en la página de ejemplo, la primera imagen de dicha página es:

   __&lt;li>&lt;img src="multimedia/galerias/IRUN120/2Alvia_120.jpg" alt="" data-description="">&lt;/li>__
   
A continuación, se puede ver el contenido html de esa página.

In [ ]:
import requests
url="https://www.vialibre-ffe.com/multi_galeria.asp?gal=524"
r = requests.get(url)
html = r.text
print(html)

Se pide:

 1. Crear un programa en __Python__ que muestre un menú con los grupos de fotos, y que pregunte al usuario que grupo de fotos quiere visitar. Se puede asociar a cada grupo un número, y solicitar al usuario que introduzca por teclado el número del grupo. A continuación, se mostrará los subgrupos de ese grupo, y se le preguntará nuevamente al usuario que subgrupo de fotos quiere procesar. Igual que antes, se puede asociar a cada subgrupo un número, y que introduzca por teclado el número del subgrupo. Como resultado se creará un directorio en el disco local para el subgrupo elegido y en el directorio se bajarán las imágenes de los trenes. Se imprimirá por pantalla las urls de las imágenes que se están bajando. Toda la información utilizada en el programa, debe ser extraida de las páginas consideradas usando BeautifulSoup[6 puntos]
 
 2. Crear un minibuscador en __Python__ que pregunte al usuario un conjunto de palabras clave, y  recorra el sitio web buscando todos los subgrupos en cuyo título aparezca alguna de las palabras claves. Como resultado debe mostrar los subgrupos encontrados, listando el nombre y la url de cada subgrupo [4 puntos]

# Normas de entrega

* Fecha tope de entrega: 18/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre WebScraping_GrupoX donde X será el número de grupo correspondiente.


In [ ]:
'''
Crear un programa en Python que muestre un menú con los grupos de fotos, 
y que pregunte al usuario que grupo de fotos quiere visitar. 
Se puede asociar a cada grupo un número, y solicitar al usuario que introduzca por teclado el número del grupo.
A continuación, se mostrará los subgrupos de ese grupo, 
y se le preguntará nuevamente al usuario que subgrupo de fotos quiere procesar. 
Igual que antes, se puede asociar a cada subgrupo un número, 
y que introduzca por teclado el número del subgrupo. 
Como resultado se creará un directorio en el disco local para el subgrupo elegido
y en el directorio se bajarán las imágenes de los trenes. 
Se imprimirá por pantalla las urls de las imágenes que se están bajando. 
Toda la información utilizada en el programa, debe ser extraida de las páginas consideradas usando BeautifulSoup[6 puntos]

'''

import urllib
import re
import os
from bs4 import BeautifulSoup

def guardafotos():

    url = "https://www.vialibre-ffe.com/multi_foto.asp?cs=mult"
    html = urllib.request.urlopen(url).read()
    bs = BeautifulSoup(html,'html.parser')

    et = bs("a")
    print("Elija el grupo de fotos que quiere visitar:\n")
    listaGrupos = []
    cont = 0
    for etiq in et:
        enlace = etiq.get("href", None)
        if "/multi_ind_fotos.asp?cat=" in enlace:
            listaGrupos.append((etiq.text, enlace))
            cont += 1
    i = 0
    while i < cont:
        print (str(i+1) + ": " + listaGrupos[i][0])
        i += 1

    num = int(input("\nIntroduzca el número asociado al grupo que desea ver: "))

    while num > len(listaGrupos) or num < 1:
        num = int(input("Introduzca el número asociado al grupo que desea ver: "))

    urlGrupo = "https://www.vialibre-ffe.com" + listaGrupos[num-1][1]
    htmlGrupo = urllib.request.urlopen(urlGrupo).read()
    bsGrupo = BeautifulSoup(htmlGrupo,'html.parser')
    etGrupo = bsGrupo('a',{"class":"tit_art_dest"})

    print("\nElija el subgrupo de fotos que quiere visitar:\n")

    listaSubgrupos = []
    cont = 0

    for etiq in etGrupo:
        enlace = etiq.get("href", None)
        listaSubgrupos.append((etiq.text, enlace))
        cont += 1
    i = 0
    while i < cont:
        print (str(i+1) + ": " + listaSubgrupos[i][0])
        i += 1

    num = int(input("\nIntroduzca el número asociado al subgrupo que desea ver: "))

    while num > len(listaSubgrupos) or num < 1:
        num = int(input("Introduzca el número asociado al subgrupo que desea ver: "))

    urlSubgrupo = "https://www.vialibre-ffe.com/" + listaSubgrupos[num-1][1]
    nombreDirectorio = listaSubgrupos[num-1][0]

    os.system('mkdir fotos') # Siempre se usa el mismo directorio, por tanto se sobreescriben fotos pasada tras pasada

    htmlFinal = urllib.request.urlopen(urlSubgrupo).read()
    bsFinal = BeautifulSoup(htmlFinal,'html.parser')
    etFinal = bsFinal('img')

    i = 1
    for etiq in etFinal:
        enlace = etiq.get("src", None)

        if "multimedia/galerias" in enlace:
            archivo=open("fotos/imagen" + str(i) + ".jpg","wb")
            imagen=urllib.request.urlopen("https://www.vialibre-ffe.com/" + str(enlace))
            while True:
                info=imagen.read(100000)
                if len(info)<1: 
                    break
                archivo.write(info)
            archivo.close()
            i += 1
        
guardafotos()        

In [ ]:
"""2.Crear un minibuscador en Python que pregunte al usuario un conjunto de palabras clave, 
y recorra el sitio web buscando todos los subgrupos en cuyo título aparezca alguna de las palabras claves. 
Como resultado debe mostrar los subgrupos encontrados, listando el nombre y la url de cada subgrupo [4 puntos]"""


import urllib
import re
from bs4 import BeautifulSoup

def buscador():

    palabras = input("Introduzca las palabras clave: ")
    palabras = palabras.upper()
    separadas = palabras.split()

    url = "https://www.vialibre-ffe.com/multi_foto.asp?cs=mult"
    html = urllib.request.urlopen(url).read()
    bs = BeautifulSoup(html,'html.parser')
    et = bs("a")

    listaGrupos = []
    cont = 0
    for etiq in et:
        enlace = etiq.get("href", None)
        if "/multi_ind_fotos.asp?cat=" in enlace:
            listaGrupos.append((etiq.text, enlace))
            cont += 1

    print("\nCoincidencias:\n")

    for texto, enlace in listaGrupos:
        url = "https://www.vialibre-ffe.com" + enlace
        html = urllib.request.urlopen(url).read()
        bs = BeautifulSoup(html,'html.parser')
        et = bs('a',{"class":"tit_art_dest"})

        for etiq in et:

            enlace = etiq.get("href", None)
            temp = etiq.text.upper()

            for elem in separadas:
                if elem in temp:
                    print(etiq.text + ", con url " + "https://www.vialibre-ffe.com/" + enlace)
                    break

buscador()
